## python script for knowing best hyper parameter's for max accuracy

In [1]:
!pip install torch torchvision torchaudio

  Using cached torchvision-0.22.1-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
  Using cached torchaudio-2.7.1-cp311-cp311-win_amd64.whl.metadata (6.6 kB)
Using cached torchvision-0.22.1-cp311-cp311-win_amd64.whl (1.7 MB)
Using cached torchaudio-2.7.1-cp311-cp311-win_amd64.whl (2.5 MB)

   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ----------------------------

In [3]:
!pip install scikit-learn

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.7 MB 3.4 MB/s eta 0:00:04
   ------ --------------------------------- 1.8/10.7 MB 4.4 MB/s eta 0:00:03
   ---------- ----------------------------- 2.9/10.7 MB 4.7 MB/s eta 0:00:02
   -------------- ------------------------- 3.9/10.7 MB 4.8 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/10.7 MB 5.1 MB/s eta 0:00:02
   ------------------------- -------------- 6.8/10.7 MB 5.4 MB/s eta 0:00:01
   ------------------------------ --------- 8.1/10.7 MB 5.5 MB/s eta 0:00:01
   ------------------------------------ --- 9.7/10.7 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 10.7/10.7 MB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   - -------------------------------------- 1.3/38.6 MB 6.7 MB/s eta 0:00:06
   -- --------------

In [4]:
# accident_classifier_gridsearch_tqdm.py

import os
import sys
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from itertools import product
from tqdm.notebook import tqdm

In [ ]:
# Paths
DATA_DIR = "unified_data"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameter grid
param_grid = {
    "model_arch": ["resnet18", "resnet34"],
    "lr": [1e-3, 1e-4],
    "batch_size": [16, 32],
    "activation": [nn.ReLU, nn.LeakyReLU]
}

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset
full_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Utility to get model
from torchvision.models import resnet18, resnet34, ResNet18_Weights, ResNet34_Weights

def get_model(arch, activation_fn):
    if arch == "resnet18":
        weights = ResNet18_Weights.DEFAULT
        model = resnet18(weights=weights)
    elif arch == "resnet34":
        weights = ResNet34_Weights.DEFAULT
        model = resnet34(weights=weights)
    else:
        raise ValueError("Unsupported architecture")

    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_features, 256),
        activation_fn(),
        nn.Dropout(0.3),
        nn.Linear(256, 2)
    )
    return model.to(DEVICE)

# Grid Search Loop
results = []
EPOCHS = 5

for arch, lr, batch_size, act_fn in product(
    param_grid["model_arch"], param_grid["lr"], param_grid["batch_size"], param_grid["activation"]):

    print(f"\n Testing: {arch}, lr={lr}, batch={batch_size}, activation={act_fn.__name__}")

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    model = get_model(arch, act_fn)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Train with tqdm progress
    for epoch in range(EPOCHS):
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        loop = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{EPOCHS}")
        for i, (images, labels) in loop:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (preds == labels).sum().item()

            avg_loss = train_loss / train_total if train_total > 0 else 0.0
            avg_acc = train_correct / train_total if train_total > 0 else 0.0
            loop.set_postfix(train_loss=avg_loss, train_acc=avg_acc)
            sys.stdout.flush()

    # Validation Accuracy and Loss
    model.eval()
    val_preds, val_labels = [], []
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

            val_loss += loss.item() * images.size(0)
            val_total += labels.size(0)
            val_correct += (preds == labels).sum().item()

    val_acc = val_correct / val_total
    avg_val_loss = val_loss / val_total
    acc = accuracy_score(val_labels, val_preds)
    results.append((arch, lr, batch_size, act_fn.__name__, acc))
    print(f" Epoch {EPOCHS} Summary: Train Acc: {avg_acc:.4f}, Train Loss: {avg_loss:.4f}, Val Acc: {val_acc:.4f}, Val Loss: {avg_val_loss:.4f}")

# Show best
results.sort(key=lambda x: x[-1], reverse=True)
print("\n Top Configurations:")
for r in results[:5]:
    print(f"Model={r[0]}, LR={r[1]}, Batch={r[2]}, Activation={r[3]}, Acc={r[4]:.4f}")